In [ ]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict
%matplotlib inline

df = pd.read_csv("salario.csv")
count = defaultdict(int)

In [ ]:
def testing():
    return df.groupby(["pais","estado","cidade","genero","data"], axis=0, level=None, as_index=False).count()

def classEquivalence(a,b,c,d,e):
    return df[(df["pais"] == a) & (df["estado"] == b) & (df["cidade"] == c) & (df["genero"] == d) & (df["data"] == e)]
    

In [ ]:
def toGeneralize(genero): 
    if genero == True: return random.choice(df.columns.drop('salario').drop('id'))
    return random.choice(df.columns.drop('salario').drop('id').drop('genero'))

def nextGeneralization(i,j, value):
    if(j == "pais"): 
        if(df.at[i,"estado"] != "*"): return nextGeneralization(i, "estado", value)
        df.at[i, j] = "*"
        if(count["lugar",i] == 2): count["lugar",i] = 3
    elif(j == "estado"):
        if(df.at[i,"cidade"] != "*"): return nextGeneralization(i, "cidade", value)
        df.at[i, j] = "*"
        if(count["lugar",i] == 1): count["lugar",i] = 2
    elif(j == "cidade"):
        df.at[i, j] = "*"
        if(count["lugar",i] == 0): count["lugar",i] = 1
    elif(j == "genero"):
        df.at[i, j] = "*"
        if(count["genero",i] == 0): count["genero",i] = 1
    elif(j == "data"):
        if(str(value)[0:5] == "**/**"): df.at[i, j] = "**/**/**"
        elif(str(value)[0:2] == "**"): df.at[i, j] = "**/**"  + str((value)[5:8])
        else:
            df.at[i, j] = "**" + str((value)[2:8])
        
        if(count["data",i] < 3): count["data",i] = count["data",i] + 1
    else:
        df.at[i, j] = "erro"
        

In [ ]:
def max_hye(col):
    if (col == 'genero'): return 1
    return 3

Na = 3 #n_atributos semi identificadores
def precision():
    summ =  0
    for index, row in df.iterrows():
        for col in df.columns.drop('id').drop('salario'):
                summ += count[col, index]/max_hye(col)

    return 1 - summ/(df.shape[0] * Na)

In [ ]:
def nextt2(qtd):
    for q in range(qtd):
        i = random.choice(df.index)
        j = toGeneralize(True)
        print(str(i) + " " + str(j))
        nextGeneralization(i, j, df.loc[i][j])
        print("----------------\n\n")

    
def nextt(k, genero):
        
        test = testing()
        test = test[test['salario'] < k]
        
        for index,aux in test.iterrows():
            a = aux['pais']
            b = aux['estado']
            c = aux['cidade']
            d = aux['genero']
            e = aux['data']
        
            
            j = toGeneralize(genero)

            for i in classEquivalence(a,b,c,d,e).index:
                print(str(i) + " " + str(j))
                nextGeneralization(i, j, aux[j])
            print("----------------\n\n")

def kanonimity(k):
    while testing()[testing()['salario'] < k].shape[0] > 0:
        qtd_restante = testing()[testing()['salario'] < k].shape[0]
        print('@@@@@@@@@@@@@@@@')
        print(qtd_restante)
        print('@@@@@@@@@@@@@@@@')
        test = testing().loc[0]
        if(qtd_restante == 1 and test['pais']=='*' and test['estado']=='*' and test['cidade']=='*'
        and test['genero']=='*' and test['data']=='**/**/**'):
            nextt2(qtd_restante)
        else:
            if(qtd_restante > 50): 
                nextt(k, False)
            else:
                nextt(k, True)

        

In [ ]:
%%time
df = pd.read_csv("salario.csv")
count = defaultdict(int)
kanonimity(2)


In [ ]:
precision() #k =2

In [ ]:
%%time
df = pd.read_csv("salario.csv")
count = defaultdict(int)
kanonimity(4)


In [ ]:
precision() #k=4

In [ ]:
%%time
df = pd.read_csv("salario.csv")
count = defaultdict(int)
kanonimity(8)


In [ ]:
precision() #k=8

In [ ]:
%%time
df = pd.read_csv("salario.csv")
count = defaultdict(int)
kanonimity(16)


In [ ]:
precision() #k=16

In [ ]:
%%time
df = pd.read_csv("salario.csv")
count = defaultdict(int)
kanonimity(64)

In [ ]:
precision() #k=64